<a href="https://colab.research.google.com/github/SCS-Technology-and-Innovation/IntroComp/blob/main/modular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modular arithmetic

The remainder of an integer division is called the *modulus*: one divides $a$ by $b$. Let's say $b$ "fits" into $a$ a total of $c$ times and what's left over is $d$: $$a = c \times b + d.$$ This means that $a \mod b \equiv d$.

In Python (as in many computational tools), the operator that does this is `%`. Python also has a specific operator for integer division: `c = a // b` combined with `d = a % b` will result in `a == c * b + d`.

In [1]:
a = 12345
b = 543

c = a // b
d = a % b
a == c * b + d

True

Though it is not obvious from the get-go, the operator `%` (called *modulo* or mod for short) comes handy quite often. For example, one can figure out whether a number $n$ is even or odd.

In [4]:
n = 87987987 // 78609 # most people cannot figure out in their heads whether the result is even or odd

if n % 2 == 0: # even numbers can be divided by two with nothing left over
  print(n, 'is even')
else:
  print(n, 'is odd')

1119 is odd


## Basic ciphers

One field of computation where *modular arithmetic* (read: math that uses $mod$ a lot) runs rampant is *cryptography*, a basic building block of information security. As an example, we will work with two fundamental algorithms: the **Diffie-Hellman key echange** and the **RSA algorithm for public-key encryption**.

### Diffie-Hellman key exchange

[Wikipedia](https://en.wikipedia.org/wiki/Diffie%E2%80%93Hellman_key_exchange) explains how it works. The below code uses the same values as the figure on the Wikipedia page of DH for the secret keys of Alice `a` and Bob `b`.


In [20]:
a = 4 # Alice chooses this and keeps it secret
b = 3 # Bob chooses this and keeps it secret

# Alice and Bob choose these together over a public channel
p = 23
g = 5

In [21]:
# Alice makes this calculation at her end
A = g**a % p

In [22]:
# Bob makes this calculation at his end
B = g**b % p

At this point, Alice sends the value of `A` to Bob on a public channel and he sends her the value of `B`.

In [23]:
print(A)
print(B)

4
10


In [27]:
# Alice makes this calculation with the B she received
sA = B**a % p

In [28]:
# Bob makes this calculation with the A he received
sB = A**b % p

In [30]:
# now, Alice and Bob hold the same value
print(sA, sB)
sA == sB
s = sA

18 18


The trick here is that, assuming all the values involved are large integers meeting certain mathematical propertied, anyone who only knows the values `p` `g` `A` and `B` cannot (without excessive computation) figure out the value of `s` since they do have knowledge of neither `a` nor `b`.

Now, note that if the numbers involved are very large, calculating the powers $g^a$, $g^b$, $B^a$ and $A^b$ is a lot of work. Not to worry, modular arithmetic comes to the rescue, hand in hand with binary representation: modular exponentiation; you can consult the details on [Wikipedia](https://en.wikipedia.org/wiki/Modular_exponentiation) for the **right-to-left binary method** (*exponentiation by squaring*).

The code is complex for a beginner to understand, but it's okay to just treat this as evidence of the magical powers of modular arithemetic and the coolness of binary numbers for now.

In [32]:
def expmod(b, e, m): # compute b**e % m
  if m == 1:
    return 0
  r = 1
  b %= m # just in case b > m
  while e > 0: # go over the binary representation of the exponent
    if e % 2 == 1: # when there are ones
      r *= b # multiply into the accumulated result
      r %= m # apply the modulus to keep things small
    e >>= 1
    b *= b # keep ramping up the squares
    b %= m # apply the modulus to keep things small
  return r

Let's test it with small numbers to check if it gives the right answer.

In [34]:
a = 67
b = 879
c = 17

normal = a**b % c
cool = expmod(a, b, c)
normal == cool

True

Then let's flex this to see how it is still very fast with big numbers.

In [43]:
a = 6790808706
b = 879507
c = 1765
expmod(a, b, c)

721

In comparison, the easy way gets super slow. If you click on the below while `a` and `b` are very large, you might want to go to the Runtime menu above and interrupt the execution

In [44]:
a**b % c

721

Now we can rewrite DH to be a very fast calculation using this beauty.

In [45]:
a = 8798776
b = 9789788
p = 7687
g = 97879890

A = expmod(g, a, p)
B = expmod(g, b, p)
expmod(B, a, p) == expmod(A, b, p)

True

## Rivest-Shamir-Adleman public-key encryption

Again, please see [Wikipedia](https://en.wikipedia.org/wiki/RSA_(cryptosystem)) for details. This is not for two individuals, but for one individual who wants to have a pair of keys: one **private** and one **public** so that when others send them messages, others use the *public* one to *cipher* what they send and nobody except the person holding the *private* they can *decipher* the messages.

In [132]:
# the person making the keys chooses two large prime numbers and keeps them secret
p = 8767867
q = 979873

Wait a second. How do we know if those numbers are *prime*? (For those unsure, an integer is prime if it is not exactly divisible by any other number than itself and one).

Note that the "exactly divisible" means "zero modulus".

For a non-prime $x$, there must exist two integers $y$ and $z$ such that $x = y \times z$ and necessarily the smaller of those, $\min \{y, z\}$ cannot exceed $\sqrt{x}$ (this is neat to reason if you are keen on math).

We can work with the above to observations to check relatively efficiently whether a given integer is prime.

In [47]:
from math import sqrt, ceil

def prime(n: int) -> bool: # being fancy here and saying that this only works for integers and will result in a truth value
  assert n > 0 # this only makes sense for strictly positive integers
  if n < 4:
    return True # 1 2 and 3 are all prime, no use analyzing
  if n % 2 == 0:
    return False # all even numbers except 2 are clearly not prime since 2 divides them
  high = sqrt(n) # the rounded-up integer of the squareroot of n is the largest possible minimum divisor
  for d in range(3, ceil(high) + 1, 2): # we try to divide with all _odd_ numbers until that upper limit
    if n % d == 0: # found a divisor, n cannot be prime
      return False
  return True

Let's test it.

In [48]:
for i in range(1, 31):
  print(i, prime(i))

1 True
2 True
3 True
4 False
5 True
6 False
7 True
8 False
9 False
10 False
11 True
12 False
13 True
14 False
15 False
16 False
17 True
18 False
19 True
20 False
21 False
22 False
23 True
24 False
25 False
26 False
27 False
28 False
29 True
30 False


Looking good. Now we can test those `p` and `q` that we needed to be prime.

In [122]:
print(prime(p), prime(q))

True True


May the RSA calculations begin.

In [133]:
n = p * q # this number will become part of the PUBLIC key
print(n)

8591396140891


The next step involves finding the greatest common denominator of $p - 1$ and $q - 1$ (as explained on the Wikipedia article for RSA linked above). There is a cool modular algorithm for that, which is even recursive: the [Euclidean algorithm](https://en.wikipedia.org/wiki/Euclidean_algorithm).

In [61]:
def gcd(a, b):
  if b == 0:
    return a
  return gcd(b, a % b)

Let's test again to see how for any numbers $x$ and $y$, $\text{gcd}(x, y)$ will always divide both $x$ and $y$ exactly.

In [62]:
x = 87987
y = 76876
z = gcd(x, y)

assert x % z == 0
assert y % z == 0

(When nothing is printed, it means the `assert` clauses were all true which is what we wanted.)

We now use the greatest common dividor to compute the [least common multiple](https://en.wikipedia.org/wiki/Least_common_multiple):

In [66]:
def lcm(a, b):
  return (a * b) // gcd(a, b)

Does this work? The resulting $z = \text{lcm}(x, y)$ should be exactly divisible by both $x$ and $y$.

In [68]:
x = 18793
y = 987
z = lcm(x, y)

assert z % x == 0
assert z % y == 0

In [134]:
l = lcm(p - 1, q - 1)
print(l)

1431897732192


The next step is picking an integer $1 < e < \ell$ for which $\text{gcd}(e, \ell) = 1$. This choice should be made at random (the next module will give ideas on how that could be emulated).

In [135]:
e = 1897
assert 1 < e
assert e < l
assert gcd(e, l) == 1

At this point, RSA turns to the great Euclides again as we need to compute $d$ as the modular inverse of $e$ in modulo $\ell$, $$ e \times d \equiv 1 \mod \ell.$$

In [88]:
def egcd(a, b):
  if a == 0:
    return b, 0, 1
  x, y, z = egcd(b % a, a)
  y, z = z - (b // a) * y, y
  return x, y, z

This one is called the [extended Euclidean algorithm](https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm) since it also gives us multiples $x$ and $y$ such that $$x \times a + y \times b = \text{gcd}(a, b)$$ instead of just the greatest common divisor itself.

In [89]:
a = 723
b = 64
c, d, e = egcd(a, b)

assert c == gcd(a, b)
assert d * a + e * b == c

This leads us to $d$ since we need $$d \times e + x \times \ell = 1$$ where we do not care at all what $x$ is.

In [136]:
one, d, discard = egcd(e, l)

assert one == 1 # gcd(e, l) should still be one
assert d * e % l == 1 # our modular inverse
if d < 0:
  d += l
print(d)

1150349047897


Now the **public** key will be $(e, n)$ and the **private** key is $(d, n)$. Nobody else should *ever* know $p$ and $q$ and even the person themselves no longer needs those two values. (The security of the method lies on good choices of $p$, $q$, and $e$.)

Now, assume that someone wants to send a message $m$ to the person who generated these keys. Assume $m < n$ (and think of the number $m$ as an encoding to some text).

In [140]:
m = 1897
assert m < n

The sender computes $m^e \mod n$ to create an *encrypted* message, call it $c$ (for *cipher*).

In [141]:
c = expmod(m, e, n)
assert c < n
print(c)

680270767256


Now only the person who knows $d$ can compute $c^d \mod n$ which should be the same as $m$ since $$(m^e)^d \equiv m \mod n$$ because $e \times d \equiv 1 \mod n$.

In [142]:
recovered = expmod(c, d, n)
print(recovered, m)

1897 1897


RSA can also be used by the key owner to sign documents using $d$ to prove to others (who have access to $e$).

In [143]:
m2 = 18989 # to sign
s = expmod(m2, d, n) # signed
v = expmod(s, e, n) # verification
v == m2

True